<a href="https://colab.research.google.com/github/fookseng/LSTM/blob/main/makeimage_fs_lstm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import datetime
import IPython
import IPython.display
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import tensorflow as tf
from google.colab import drive
from google.colab.patches import cv2_imshow # for image display
from tensorflow.keras.models import Sequential, model_from_json, load_model
from tensorflow.keras.layers import InputLayer, Dense, Activation, Flatten, LeakyReLU, ReLU, PReLU, Dropout, BatchNormalization, LSTM
from tensorflow.keras.optimizers import Adam, SGD, Nadam, Adadelta 
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
from PIL import Image as im
from keras.preprocessing.image import image
import shutil
import imageio
from numpy import array
from sklearn.preprocessing import MinMaxScaler

In [ ]:
# Check tensorflow version
print(tf.__version__)

2.6.0


In [ ]:
# Mount Google Drive to access data
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


# *Functions*

In [ ]:
def create_data(x, y, i):
  '''
  We will combine dataset from 2011-2017 and 2019 to use as train and valid dataset (8:2).
  2018 dataset will be  used as test set.
  > T2 is time(T-2), T1 is time(T-1), and T is current time T.
  '''
  # Pixel dataset
  images_T2 = np.load(path +'/image_array_0.npy')
  images_T1 = np.load(path +'/image_array_1.npy')
  images_T = np.load(path +'/image_array_2.npy')
  # Flow dataset
  flow_T2 = np.load(path +'/flow_array_0.npy')
  flow_T1 = np.load(path +'/flow_array_1.npy')
  flow_T = np.load(path +'/flow_array_2.npy')
  # Tide dataset
  tide_T2 = np.load(path +'/delta_tide_array_0.npy')
  tide_T1 = np.load(path +'/delta_tide_array_1.npy')
  tide_T = np.load(path +'/delta_tide_array_2.npy')
  
  print("Creating dataset on point:", x,",", y)
  pixel_data_train.append(images_T2[i][y][x])
  pixel_data_train.append(images_T1[i][y][x])
  pixel_data_train.append(images_T[i][y][x])
  flow_data_train.append(flow_T2[i][0])
  flow_data_train.append(flow_T1[i][0])
  flow_data_train.append(flow_T[i][0])
  tide_data_train.append(tide_T2[i][0])
  tide_data_train.append(tide_T1[i][0])
  tide_data_train.append(tide_T[i][0])
  pixel1_data_train.append(images_T2[i][y-1][x+1])
  pixel1_data_train.append(images_T1[i][y-1][x+1])
  pixel1_data_train.append(images_T[i][y-1][x+1])
  pixel2_data_train.append(images_T2[i][y][x+1])
  pixel2_data_train.append(images_T1[i][y][x+1])
  pixel2_data_train.append(images_T[i][y][x+1])
  pixel3_data_train.append(images_T2[i][y+1][x+1])
  pixel3_data_train.append(images_T1[i][y+1][x+1])
  pixel3_data_train.append(images_T[i][y+1][x+1])
  pixel4_data_train.append(images_T2[i][y-1][x])
  pixel4_data_train.append(images_T1[i][y-1][x])
  pixel4_data_train.append(images_T[i][y-1][x])
  pixel5_data_train.append(images_T2[i][y+1][x])
  pixel5_data_train.append(images_T1[i][y+1][x])
  pixel5_data_train.append(images_T[i][y+1][x])
  pixel6_data_train.append(images_T2[i][y-1][x-1])
  pixel6_data_train.append(images_T1[i][y-1][x-1])
  pixel6_data_train.append(images_T[i][y-1][x-1])
  pixel7_data_train.append(images_T2[i][y][x-1])
  pixel7_data_train.append(images_T1[i][y][x-1])
  pixel7_data_train.append(images_T[i][y][x-1])
  pixel8_data_train.append(images_T2[i][y+1][x-1])
  pixel8_data_train.append(images_T1[i][y+1][x-1])
  pixel8_data_train.append(images_T[i][y+1][x-1])
  return tide_T.size

In [ ]:
def multi_data_prep(dataset, target):
     X = []
     y = []
     for i in range(0, len(dataset), 3):
         X.append(dataset[i:i+2])
         y.append(target[i+2])
     return np.array(X), np.array(y) 

# *Main function*

In [ ]:
if __name__ == '__main__':
  for z in range(83, 300, 1):
    path = '/content/gdrive/Shareddrives/namr - water quality forecast/20211018_70%up_continuous_for_3/CHL'
    savePath = '/content/gdrive/Shareddrives/namr - water quality forecast/fs/model2/'
    savePath2 = '/content/gdrive/Shareddrives/namr - water quality forecast/fs/'
    mode = 3
    model_type = "multivariate"
    state_name ="stateful"
    model_name = "chl"
    k = z
    previous = 0
    arr = []
    array1 = np.load(path +'/image_array_0.npy')
    array2 = np.load(path +'/image_array_1.npy')
    array3 = np.load(path +'/image_array_2.npy')
    arrayPredict = np.load(path +'/image_array_2.npy')
    for i in range(89):
      for j in range(74):
        if (i>=63 and i <= 78) and (j>=11 and j <= 16):
          x_pos = j
          y_pos = i
          ### Define some variables
          pixel_data_train = []
          flow_data_train = []
          tide_data_train = []
          pixel1_data_train =[]
          pixel2_data_train =[]
          pixel3_data_train =[]
          pixel4_data_train =[]
          pixel5_data_train =[]
          pixel6_data_train =[]
          pixel7_data_train =[]
          pixel8_data_train =[]

          ### Create data and save as pandas DataFrame
          m = create_data(x_pos, y_pos, k)
          dic_train = {'pixel': pixel_data_train, 'pixel1': pixel1_data_train, 'pixel2': pixel2_data_train, 'pixel3': pixel3_data_train, 'pixel4': pixel4_data_train, 'pixel5': pixel5_data_train, 'pixel6': pixel6_data_train, 'pixel7': pixel7_data_train, 'pixel8': pixel8_data_train, 'flow': flow_data_train, 'tide': tide_data_train}
          df_train = pd.DataFrame(dic_train)  
          features_data = df_train[["pixel", "pixel1", "pixel2", "pixel3", "pixel4", "pixel5", "pixel6", "pixel7", "pixel8", "tide"]].to_numpy()
          target_data = df_train[["pixel"]].to_numpy()
          x_sampled, y_sampled = multi_data_prep(features_data, target_data)
          x_sampled = x_sampled.reshape((x_sampled.shape[0], x_sampled.shape[1], 10))
          filepath = savePath +'/model_'+ model_type+ '_'+ state_name+ '_'+ "mode"+ str(mode)+ '_'+ str(j)+ '_'+ str(i)+ '_' +model_name+'.h5'
          model = load_model(filepath)
          testX = x_sampled

          predict_value = model.predict(testX, batch_size=1)
          if predict_value:
            previous = predict_value
          else:
            #predict_value = previous
            arr.append([j,i])
          arrayPredict[k][i][j] = predict_value[0][0]
          #print("P: "+str(predict_value[0][0]))
        else:  
          arrayPredict[k][i][j] = (0.3*array1[k][i][j] + 0.7*array2[k][i][j])
          #print("R: "+str(array3[k][i][j]))
    
    # imageio.imwrite(savePath2 + 'image/'+'image'+'_newt2.png', array1[k])
    # imageio.imwrite(savePath2 + 'image/'+'image'+'_newt1.png', array2[k])
    # imageio.imwrite(savePath2 + 'image/'+'image'+'_newt.png', array3[k])
    imageio.imwrite(savePath2 + 'image/'+'image'+str(k)+'_newPre.png', arrayPredict[k])
    print("ENDO")
    print(arr)

Creating dataset on point: 11 , 63
Creating dataset on point: 12 , 63
Creating dataset on point: 13 , 63
Creating dataset on point: 14 , 63
Creating dataset on point: 15 , 63
Creating dataset on point: 16 , 63
Creating dataset on point: 11 , 64
Creating dataset on point: 12 , 64
Creating dataset on point: 13 , 64
Creating dataset on point: 14 , 64
Creating dataset on point: 15 , 64
Creating dataset on point: 16 , 64
Creating dataset on point: 11 , 65
Creating dataset on point: 12 , 65
Creating dataset on point: 13 , 65
Creating dataset on point: 14 , 65
Creating dataset on point: 15 , 65
Creating dataset on point: 16 , 65
Creating dataset on point: 11 , 66
Creating dataset on point: 12 , 66
Creating dataset on point: 13 , 66
Creating dataset on point: 14 , 66
Creating dataset on point: 15 , 66
Creating dataset on point: 16 , 66
Creating dataset on point: 11 , 67
Creating dataset on point: 12 , 67
Creating dataset on point: 13 , 67
Creating dataset on point: 14 , 67
Creating dataset on 

ENDO
[[14, 67], [13, 68], [14, 68], [11, 69], [13, 70], [13, 74], [15, 78]]
Creating dataset on point: 11 , 63
Creating dataset on point: 12 , 63
Creating dataset on point: 13 , 63
Creating dataset on point: 14 , 63
Creating dataset on point: 15 , 63
Creating dataset on point: 16 , 63
Creating dataset on point: 11 , 64
Creating dataset on point: 12 , 64
Creating dataset on point: 13 , 64
Creating dataset on point: 14 , 64
Creating dataset on point: 15 , 64
Creating dataset on point: 16 , 64
Creating dataset on point: 11 , 65
Creating dataset on point: 12 , 65
Creating dataset on point: 13 , 65
Creating dataset on point: 14 , 65
Creating dataset on point: 15 , 65
Creating dataset on point: 16 , 65
Creating dataset on point: 11 , 66
Creating dataset on point: 12 , 66
Creating dataset on point: 13 , 66
Creating dataset on point: 14 , 66
Creating dataset on point: 15 , 66
Creating dataset on point: 16 , 66
Creating dataset on point: 11 , 67
Creating dataset on point: 12 , 67
Creating datas

ENDO
[[13, 68], [14, 68], [11, 69], [13, 70], [15, 78]]
Creating dataset on point: 11 , 63
Creating dataset on point: 12 , 63
Creating dataset on point: 13 , 63
Creating dataset on point: 14 , 63
Creating dataset on point: 15 , 63
Creating dataset on point: 16 , 63
Creating dataset on point: 11 , 64
Creating dataset on point: 12 , 64
Creating dataset on point: 13 , 64
Creating dataset on point: 14 , 64
Creating dataset on point: 15 , 64
Creating dataset on point: 16 , 64
Creating dataset on point: 11 , 65
Creating dataset on point: 12 , 65
Creating dataset on point: 13 , 65
Creating dataset on point: 14 , 65
Creating dataset on point: 15 , 65
Creating dataset on point: 16 , 65
Creating dataset on point: 11 , 66
Creating dataset on point: 12 , 66
Creating dataset on point: 13 , 66
Creating dataset on point: 14 , 66
Creating dataset on point: 15 , 66
Creating dataset on point: 16 , 66
Creating dataset on point: 11 , 67
Creating dataset on point: 12 , 67
Creating dataset on point: 13 , 67

ENDO
[[13, 68], [11, 69], [15, 78]]
Creating dataset on point: 11 , 63
Creating dataset on point: 12 , 63
Creating dataset on point: 13 , 63
Creating dataset on point: 14 , 63
Creating dataset on point: 15 , 63
Creating dataset on point: 16 , 63
Creating dataset on point: 11 , 64
Creating dataset on point: 12 , 64
Creating dataset on point: 13 , 64
Creating dataset on point: 14 , 64
Creating dataset on point: 15 , 64
Creating dataset on point: 16 , 64
Creating dataset on point: 11 , 65
Creating dataset on point: 12 , 65
Creating dataset on point: 13 , 65
Creating dataset on point: 14 , 65
Creating dataset on point: 15 , 65
Creating dataset on point: 16 , 65
Creating dataset on point: 11 , 66
Creating dataset on point: 12 , 66
Creating dataset on point: 13 , 66
Creating dataset on point: 14 , 66
Creating dataset on point: 15 , 66
Creating dataset on point: 16 , 66
Creating dataset on point: 11 , 67
Creating dataset on point: 12 , 67
Creating dataset on point: 13 , 67
Creating dataset on

ENDO
[[13, 68], [11, 69], [15, 78]]
Creating dataset on point: 11 , 63
Creating dataset on point: 12 , 63
Creating dataset on point: 13 , 63
Creating dataset on point: 14 , 63
Creating dataset on point: 15 , 63
Creating dataset on point: 16 , 63
Creating dataset on point: 11 , 64
Creating dataset on point: 12 , 64
Creating dataset on point: 13 , 64
Creating dataset on point: 14 , 64
Creating dataset on point: 15 , 64
Creating dataset on point: 16 , 64
Creating dataset on point: 11 , 65
Creating dataset on point: 12 , 65
Creating dataset on point: 13 , 65
Creating dataset on point: 14 , 65
Creating dataset on point: 15 , 65
Creating dataset on point: 16 , 65
Creating dataset on point: 11 , 66
Creating dataset on point: 12 , 66
Creating dataset on point: 13 , 66
Creating dataset on point: 14 , 66
Creating dataset on point: 15 , 66
Creating dataset on point: 16 , 66
Creating dataset on point: 11 , 67
Creating dataset on point: 12 , 67
Creating dataset on point: 13 , 67
Creating dataset on

...

ENDO
[[15, 63], [13, 68], [11, 69], [15, 73], [15, 78]]
Creating dataset on point: 11 , 63
Creating dataset on point: 12 , 63
Creating dataset on point: 13 , 63
Creating dataset on point: 14 , 63
Creating dataset on point: 15 , 63
Creating dataset on point: 16 , 63
Creating dataset on point: 11 , 64
Creating dataset on point: 12 , 64
Creating dataset on point: 13 , 64
Creating dataset on point: 14 , 64
Creating dataset on point: 15 , 64
Creating dataset on point: 16 , 64
Creating dataset on point: 11 , 65
Creating dataset on point: 12 , 65
Creating dataset on point: 13 , 65
Creating dataset on point: 14 , 65
Creating dataset on point: 15 , 65
Creating dataset on point: 16 , 65
Creating dataset on point: 11 , 66
Creating dataset on point: 12 , 66
Creating dataset on point: 13 , 66
Creating dataset on point: 14 , 66
Creating dataset on point: 15 , 66
Creating dataset on point: 16 , 66
Creating dataset on point: 11 , 67
Creating dataset on point: 12 , 67
Creating dataset on point: 13 , 67

ENDO
[[15, 63], [13, 68], [11, 69], [15, 73], [15, 78]]
Creating dataset on point: 11 , 63
Creating dataset on point: 12 , 63
Creating dataset on point: 13 , 63
Creating dataset on point: 14 , 63
Creating dataset on point: 15 , 63
Creating dataset on point: 16 , 63
Creating dataset on point: 11 , 64
Creating dataset on point: 12 , 64
Creating dataset on point: 13 , 64
Creating dataset on point: 14 , 64
Creating dataset on point: 15 , 64
Creating dataset on point: 16 , 64
Creating dataset on point: 11 , 65
Creating dataset on point: 12 , 65
Creating dataset on point: 13 , 65
Creating dataset on point: 14 , 65
Creating dataset on point: 15 , 65
Creating dataset on point: 16 , 65
Creating dataset on point: 11 , 66
Creating dataset on point: 12 , 66
Creating dataset on point: 13 , 66
Creating dataset on point: 14 , 66
Creating dataset on point: 15 , 66
Creating dataset on point: 16 , 66
Creating dataset on point: 11 , 67
Creating dataset on point: 12 , 67
Creating dataset on point: 13 , 67

ENDO
[[15, 63], [13, 68], [11, 69], [15, 73], [11, 75], [15, 78]]
Creating dataset on point: 11 , 63
Creating dataset on point: 12 , 63
Creating dataset on point: 13 , 63
Creating dataset on point: 14 , 63
Creating dataset on point: 15 , 63
Creating dataset on point: 16 , 63
Creating dataset on point: 11 , 64
Creating dataset on point: 12 , 64
Creating dataset on point: 13 , 64
Creating dataset on point: 14 , 64
Creating dataset on point: 15 , 64
Creating dataset on point: 16 , 64
Creating dataset on point: 11 , 65
Creating dataset on point: 12 , 65
Creating dataset on point: 13 , 65
Creating dataset on point: 14 , 65
Creating dataset on point: 15 , 65
Creating dataset on point: 16 , 65
Creating dataset on point: 11 , 66
Creating dataset on point: 12 , 66
Creating dataset on point: 13 , 66
Creating dataset on point: 14 , 66
Creating dataset on point: 15 , 66
Creating dataset on point: 16 , 66
Creating dataset on point: 11 , 67
Creating dataset on point: 12 , 67
Creating dataset on poin

ENDO
[[15, 63], [13, 68], [11, 69], [15, 78]]
Creating dataset on point: 11 , 63
Creating dataset on point: 12 , 63
Creating dataset on point: 13 , 63
Creating dataset on point: 14 , 63
Creating dataset on point: 15 , 63
Creating dataset on point: 16 , 63
Creating dataset on point: 11 , 64
Creating dataset on point: 12 , 64
Creating dataset on point: 13 , 64
Creating dataset on point: 14 , 64
Creating dataset on point: 15 , 64
Creating dataset on point: 16 , 64
Creating dataset on point: 11 , 65
Creating dataset on point: 12 , 65
Creating dataset on point: 13 , 65
Creating dataset on point: 14 , 65
Creating dataset on point: 15 , 65
Creating dataset on point: 16 , 65
Creating dataset on point: 11 , 66
Creating dataset on point: 12 , 66
Creating dataset on point: 13 , 66
Creating dataset on point: 14 , 66
Creating dataset on point: 15 , 66
Creating dataset on point: 16 , 66
Creating dataset on point: 11 , 67
Creating dataset on point: 12 , 67
Creating dataset on point: 13 , 67
Creating 

ENDO
[[15, 63], [13, 68], [11, 69], [13, 74], [15, 78]]
Creating dataset on point: 11 , 63
Creating dataset on point: 12 , 63
Creating dataset on point: 13 , 63
Creating dataset on point: 14 , 63
Creating dataset on point: 15 , 63
Creating dataset on point: 16 , 63
Creating dataset on point: 11 , 64
Creating dataset on point: 12 , 64
Creating dataset on point: 13 , 64
Creating dataset on point: 14 , 64
Creating dataset on point: 15 , 64
Creating dataset on point: 16 , 64
Creating dataset on point: 11 , 65
Creating dataset on point: 12 , 65
Creating dataset on point: 13 , 65
Creating dataset on point: 14 , 65
Creating dataset on point: 15 , 65
Creating dataset on point: 16 , 65
Creating dataset on point: 11 , 66
Creating dataset on point: 12 , 66
Creating dataset on point: 13 , 66
Creating dataset on point: 14 , 66
Creating dataset on point: 15 , 66
Creating dataset on point: 16 , 66
Creating dataset on point: 11 , 67
Creating dataset on point: 12 , 67
Creating dataset on point: 13 , 67